# UNDER CONSTRUCTION

<img src="../../images/TGC_trans.png" alt="Enexis JADS" style="width:15%; float:right">

# Description

- 2 EVSE's with a fixed power delivery of 7 Kw
- max total power consumption constrainted by Enexis = 10 Kw
- EV requests a constant 10 Kwh
- 
- 

## Purpose

- Showcase that the monitor shows the power consumption of 7 Kw in time when 
  interarrival time = 10/7
- Showcase that power consumption falls to 0 when iat > 10/7, e.g. 14/7

## Visualization

<img src="images/fig_3.png" width="40%"/>
<img src="images/demo.png" width="30%"/>


# Simulation function for the e_parking

In [12]:
import salabim as sim
import pandas as pd

# ------------------------------------------------------------
# Simulation code for D/D/1
# ------------------------------------------------------------
# function to run simulation
def sim_e_parking(
    inter_arr_time_distr,  # inter-arrival time distribution
    energy_request_distr,  # energy request distribution
    fixed_utilization,
    number_of_EVs,
    number_of_PARKING_LOTs,
    number_of_EVSEs,  # number of EVSE's
    max_kw_of_EVSE,
    enexis_kw,
    sim_time,
    time_unit,
    random_seed,
    run,
):
    # ------------------------------------------------------------
    cnv_hr_to_mins = 60

    # Generator which creates EV's up to a certain number
    class EV_Generator(sim.Component):
        # setup method is called when the component is created
        # and is used to initialize the component
        # switch off monitoring for mode and status
        def setup(self, number_of_EVs):
            self.mode.monitor(False)
            self.status.monitor(False)
            self.no_of_ev = number_of_EVs

        def process(self):
            ev = 1
            while ev <= self.no_of_ev:
                EV()
                iat = inter_arr_time_distr.sample()
                if fixed_utilization:
                    iat = iat / number_of_EVSEs
                self.hold(iat)
                ev += 1

    class EV(sim.Component):
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)
            # energy request in kwh
            self.kwh_req = energy_request_distr.sample()
            self.stay = 100
            self.kwh_charged = 0

        def process(self):
            self.enter(waitingline)
            for PLOT in e_parking_plots:
                if PLOT.ispassive():
                    PLOT.activate()
                    break  # activate at most one parking lot
            self.passivate()
            print(f"EV: {self.name()} charged: {self.kwh_charged} charged at: {app.now()}")

    class PLOT(sim.Component):
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)

            self.length = sim.Monitor(
                name="length", monitor=True, level=True, type="int32"
            )
            self.length_of_stay = sim.Monitor(
                name="length_of_stay", monitor=True, level=False, type="float"
            )
            self.length_of_charge = sim.Monitor(
                name="length_of_charge", monitor=True, level=False, type="float"
            )
            self.power_mon = sim.Monitor(
                name="power.", monitor=True, level=True, type="float"
            )
            
            if EVSE_POOL.available_quantity() > 0:
                self.request(EVSE_POOL)
                self.evse = True
            else:
                self.evse = False
            self.power = max_kw_of_EVSE

        def process(self):
            while True:
                # get an ev from the queue
                self.length.tally(0)
                while len(waitingline) == 0:
                    self.passivate()
                self.ev = waitingline.pop()
                self.length.tally(1)
                if self.evse:
                  charging_time = self.ev.kwh_req / self.power
                  # charging 
                  self.length_of_charge.tally(charging_time)
                  self.power_mon.tally(self.power)
                  self.ev.kwh_charged = min(charging_time, self.ev.stay) * self.power
                  self.length_of_stay.tally(self.ev.stay)
                self.hold(self.ev.stay)
                self.ev.activate()
                self.passivate()

    # class EVSE(sim.Component):
    #     def setup(self, max_kw):
    #         self.mode.monitor(False)
    #         self.status.monitor(False)
    #         self.max_kw = max_kw
    #         self.ev = None

    #         self.length = sim.Monitor(
    #             name="length", monitor=True, level=True, type="int32"
    #         )
    #         self.length_of_stay = sim.Monitor(
    #             name="length_of_stay", monitor=True, level=False, type="float"
    #         )
    #         self.power_mon = sim.Monitor(
    #             name="power.", monitor=True, level=True, type="float"
    #         )

    #     def process(self):
    #         while True:
    #             self.length.tally(1)
    #             self.power = self.max_kw
    #             print(
    #                 f"energy requested: {self.ev.kwh_req}, power delivered: {self.power}"
    #             )
  

    # ------------------------------------------------------------
    # https://www.salabim.org/manual/Reference.html#environment
    app = sim.App(
        trace=False,  # defines whether to trace or not
        random_seed=random_seed,  # if “*”, a purely random value (based on the current time)
        time_unit=time_unit,  # defines the time unit used in the simulation
        name="Charging Station",  # name of the simulation
        do_reset=True,  # defines whether to reset the simulation when the run method is called
        yieldless=True,  # defines whether the simulation is yieldless or not
    )

    # Instantiate and activate the client generator
    EV_Generator(name="Electric Vehicles Generator", number_of_EVs=number_of_EVs)

    # Create Queue and set monitor to stats_only
    # https://www.salabim.org/manual/Queue.html
    waitingline = sim.Queue(name="Waiting EV's", monitor=True)
    # waitingline.length_of_stay.monitor(value=True)
    waitingline.length.reset_monitors(stats_only=True)
    waitingline.length_of_stay.reset_monitors(stats_only=True)

    # Create environment and start processes
    # e_parking_evse = [EVSE(max_kw = max_kw_of_EVSE) for _ in range(number_of_EVSEs)]
    EVSE_POOL = sim.Resource("EVSEs", capacity=number_of_EVSEs)

    # Instantiate the EVSE's, list comprehension
    e_parking_plots = [PLOT() for _ in range(number_of_PARKING_LOTs)]

    # app.AnimateMonitor(
    #     e_parking_evse[0].power_mon,
    #     x=100,
    #     y=550,
    #     width=890,
    #     height=300,
    #     vertical_scale=25,
    #     horizontal_scale=100,
    #     linewidth=3,
    #     linecolor="green",
    #     title="Power",
    #     labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    #     nowcolor="red",
    # )
    # app.animate(True)

    # Execute Simulation
    app.run(till=sim_time)

    # Calculate aggregate statistics
    # total_evse_stay = sum(x.length_of_stay for x in e_parking_evse).mean()
    # total_evse_lngt = sum(x.length for x in e_parking_evse).mean()

    # waitingline.mode.print_histogram(values=True)

    lmbda = cnv_hr_to_mins / inter_arr_time_distr.mean()
    if fixed_utilization:
        lmbda = lmbda * number_of_EVSEs

    # Return results
    return {
        "run": run,
        "lambda": lmbda,
        "mu": cnv_hr_to_mins / energy_request_distr.mean(),
        "c": number_of_EVSEs,
        # "RO": total_evse_lngt / number_of_EVSEs,
        "P0": 0,
        "Lq": waitingline.length.mean(),
        "Wq": waitingline.length_of_stay.mean(),
        # "Ls": total_evse_lngt + waitingline.length.mean(),
        # "Ws": total_evse_stay + waitingline.length_of_stay.mean(),
    }

# User parameters

In [13]:
# user parameters
ev_arrival_time = 10 / 7  # hours between two arrivals
energy__request = 10  # KW requested per EV

# D/D/c
inter_arr_time_distr = sim.Uniform(ev_arrival_time, ev_arrival_time)
energy_request_distr = sim.Uniform(energy__request, energy__request)
fixed_utilization = True

# Electric Vehicles
number_of_EVs = 2

# number of parking lots
number_of_PARKING_LOTs = 1

# EVSE's
number_of_EVSEs = 1
max_kw_of_EVSE = 7

# Capacity Enexis
enexis_kw = 10

# simulation parameters
sim_time = 1000 #None
time_unit = "hours"
random_seed = "*"
run = 1

# Run

In [14]:
result = sim_e_parking(
    inter_arr_time_distr,  # inter-arrival time distribution
    energy_request_distr,  # energy request distribution
    fixed_utilization,    
    number_of_EVs,
    number_of_PARKING_LOTs,
    number_of_EVSEs,  # number of EVSE's
    max_kw_of_EVSE,
    enexis_kw,
    sim_time,
    time_unit,
    random_seed,
    run,
)

print(pd.DataFrame(result, index=[0]))

EV: ev.1 charged: 10.0 charged at: 100.0
   run  lambda   mu  c  P0        Lq   Wq
0    1    42.0  6.0  1   0  0.998571  0.0
